In [0]:
import sys
sys.path.append('./keras-spp/')
from spp.SpatialPyramidPooling import SpatialPyramidPooling

from keras.models import load_model
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Dropout, Flatten, GlobalAveragePooling2D
from keras.optimizers import SGD
from keras.callbacks import ReduceLROnPlateau
import numpy as np
import glob
import os
from scipy.misc import imread, imresize
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
from skimage.color import rgb2gray
from keras.utils import np_utils
import pywt
import matplotlib.pyplot as plt
import cv2

#### Load Data

In [0]:
master_folder = '../GrantData401Project4/PhotosDataset/'

imgs = []
labels = []
for photo in os.listdir(master_folder+'/Alex'):
    img = imread(master_folder+'/Alex/'+photo)
    imgs.append(cv2.Canny(img, 300,400))
    labels.append('Alex')
for photo in os.listdir(master_folder+'/Hunter'):
    img = imread(master_folder+'/Hunter/'+photo)
    imgs.append(cv2.Canny(img, 300,400))
    labels.append('Hunter')

#### Massage Data

In [0]:
# Take out alpha component of image
imgs = [img[:,:,[0,1,2]] for img in imgs]

labels = [[1,0] if label is 'Hunter' else [0,1] for label in labels]

x_train, x_test, y_train, y_test = train_test_split(imgs, labels, test_size=.5,
                                                    stratify=labels)

In [0]:
# Use a generator to feed data because data images are of different dimensions so a numpy array cant
# be constructed
def generator(x, y):
    while True:
        for i,img in enumerate(x):
            yield np.expand_dims(img,axis=0), np.expand_dims(y[i],axis=0)
            
gen = generator(x_train, y_train)
val_gen = generator(x_test,y_test)
num_channels = 1
num_classes = 2

### Model with no transfer learning

In [0]:
model = Sequential()

# Note that we leave the image size as None to allow multiple image sizes
model.add(Conv2D(32, (3, 3), padding='same',
                 activation='relu',input_shape=(None, None,num_channels),
                name='input_layer'))
model.add(Conv2D(32, (3, 3),activation='relu',name='conv2'))
model.add(MaxPooling2D(pool_size=(2, 2), name='maxpool1'))
          
model.add(Conv2D(64, (3, 3), padding='same',activation='relu',name='conv3'))
model.add(Conv2D(64, (3, 3),name='conv4'))

# Spatial Pooling layer to deal with differing image sizes
model.add(SpatialPyramidPooling([1, 2, 4]))

# Classification layer
model.add(Dense(num_classes, activation='softmax'))

callbacks = []
callbacks.append(ReduceLROnPlateau(monitor='val_loss', factor=0.5,
                                   patience=15, verbose=1, mode='auto',
                                   min_delta=0.0001, cooldown=0, min_lr=0))
opt = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])


epochs = 10
# Fit model
hist = model.fit_generator(generator=gen,
                steps_per_epoch = len(x_train),
                epochs=epochs,validation_data=val_gen,
                validation_steps=len(x_test))




### Utilize transfer learning

In [0]:
filepath = "caltech_101_canny_edge_channel.h5"
trans_model = load_model(filepath)

# Allow no weight adjustments for the pretrained layers
for layer in trans_model.layers:
    layer.trainable = False

    
# Spatial Pooling layer to deal with differing image sizes
trans_model.add(SpatialPyramidPooling([1, 2, 4]))
# Classification Layer
trans_model.add(Dense(num_classes, activation='softmax'))

In [0]:
callbacks = []
callbacks.append(ReduceLROnPlateau(monitor='val_loss', factor=0.5,
                                   patience=15, verbose=1, mode='auto',
                                   min_delta=0.0001, cooldown=0, min_lr=0))
opt = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
trans_model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])


epochs = 10
# Fit model
hist = trans_model.fit_generator(generator=gen,
                steps_per_epoch = len(x_train),
                epochs=epochs,validation_data=val_gen,
                validation_steps=len(x_test))

Evaluate

In [0]:
trans_pred = []
reg_pred= []
for img in x_test:
    trans_pred.append(trans_model.predict(np.expand_dims(img, axis=0)))
    reg_pred.append(model.predict(np.expand_dims(img, axis=0)))

reg_pred = ["Hunter" if x[0][0] > x[0][1] else "Alex" for x in reg_pred]
trans_pred = ["Hunter" if x[0][0] > x[0][1] else "Alex" for x in trans_pred]

y_test = ["Hunter" if x[0] > x[1] else "Alex" for x in y_test]

In [0]:
from sklearn.metrics import classification_report
print("Model with No transfer learning")
print(classification_report(y_test, reg_pred))
print("Model with transfer learning")
print(classification_report(y_test, trans_pred))